In [1]:

from pyspark import SparkConf, SparkContext
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.regression import LabeledPoint
from numpy import array

In [2]:

conf = SparkConf().setMaster('local').setAppName('SparkDecisionTree')
sc = SparkContext(conf = conf)

In [3]:

def binary(YN):
    if (YN == 'Y'):
        return 1
    else:
        return 0

In [4]:

def map_education(degree):
    if degree == 'BS':
        return 1
    if degree == 'MS':
        return 2
    if degree == 'PhD':
        return 3
    else:
        return 0

In [ ]:

def_create_labeled_points(fields):
    